In [1]:
from __future__ import print_function

import os
import pickle
import sys
sys.path.append('../Load_data')
sys.path.append('../Sentiment')
import read_news
import TwitterAxe as Twa
import numpy as np
import pandas as pd
from NewsAxe import multistocks_score, index_score, buy_sell_signal
from datetime import date
from dateutil.rrule import rrule, DAILY
import read_news

In [2]:
df = pd.read_csv('../Load_data/sentiscore_au.csv', header = 0)

In [4]:
df.describe()

,Unnamed: 0,sentiment
count,3516.000000,3516.000000
mean,1757.500000,0.096686
std,1015.126101,0.238877
min,0.000000,-0.844000
25%,878.750000,0.000000
50%,1757.500000,0.103000
75%,2636.250000,0.254250
max,3515.000000,0.840000


In [2]:
def signal_index(date, key_list):
    # generate the singal for multiple stocks, using the functions in NewsAxe
    scores = multistocks_score(date, key_list)
    index_s = index_score(scores)
    signal = buy_sell_signal(index_s)
    return signal

In [3]:
def get_key_list(element_list):
    # generate a list of lists 
    key_list = []
    for element in element_list:
        key_list_single = []
        key_list_single.append((element))
        key_list.append(key_list_single)
    return key_list    

In [5]:
def check_amounts(date, key_list):
    read = read_news.read_reuters(file_date = date)
    key_string = read.locate_key(key_list)
    return key_string
    

In [86]:
def run_au():
    start_day = date(2007, 1, 1)
    end_day = date(2016, 8, 16)

    element_list = ['australia']    
    key_list = element_list

    sentiment_list = []
    date_list = []

    for dt in rrule(DAILY, dtstart=start_day, until=end_day):
        date_trading = dt.strftime("%Y%m%d")
        read = read_news.read_reuters(file_date = date_trading)
        key_string = read.locate_key(key_list)
        score = Twa.sentimentScore(key_string)
        date_list.append(dt.strftime("%Y-%m-%d"))
        sentiment_list.append(score)       
        
    df_trade = pd.DataFrame({'date':date_list, 'sentiment':sentiment_list})    
    
    return df_trade

In [87]:
if __name__ == '__main__': 
    df_trade_check = run_au()
    #df_trade_check = run_check_amounts()

There is no available news file on '2011'.
There is no available news file on '2015'.


In [67]:
def run_check_amounts():
    start_day = date(2007, 1, 1)
    end_day = date(2007, 5, 1)

    df_sp = pd.read_csv('../Load_data/asx200_elements.csv', header = 0)
    element_list = ['australia']
    #element_list = np.array(df_sp['Company'])    
    key_list = element_list

    amount_list = []
    date_list = []

    for dt in rrule(DAILY, dtstart=start_day, until=end_day):
        date_trading = dt.strftime("%Y%m%d")
        length_key = check_amounts(date_trading, element_list)
        date_list.append(dt.strftime("%Y-%m-%d"))
        amount_list.append(length_key)       
        
    df_trade = pd.DataFrame({'date':date_list, 'amount':amount_list})    
    
    return df_trade

In [89]:
df_trade_check.to_csv('../Load_data/sentiscore_au.csv', header = True)

In [58]:
def run_sp500():
    start_day = date(2007, 1, 1)
    end_day = date(2007, 3, 1)

    df_sp = pd.read_csv('../Load_data/top_sp500.csv', header = 0)
    element_list = ['australia']    
    key_list = get_key_list(element_list)

    signal_list = []
    date_list = []

    for dt in rrule(DAILY, dtstart=start_day, until=end_day):
        date_trading = dt.strftime("%Y%m%d")
        signal= signal_index(date_trading, key_list)
        date_list.append(dt.strftime("%Y-%m-%d"))
        signal_list.append(signal)       
        
    df_trade = pd.DataFrame({'date':date_list, 'signal':signal_list})    
    
    return df_trade